In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#/content/drive/MyDrive/

Mounted at /content/drive


In [ ]:
!pip install rouge-score sacrebleu

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 4.3 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=23b3a520ccd134afe04adf2d398e63cb95c29a571d75f3c3f6807fbfdd55fa44
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


# Import Thư Viện

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm
import pandas as pd
from rouge_score import rouge_scorer
import sacrebleu

# Load BartPho for Summary model

In [ ]:
# Kiểm tra xem có GPU không
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Tải mô hình BARTpho-syllable cho nhiệm vụ tóm tắt văn bản tiếng Việt
model_name = "vinai/bartpho-syllable"  # Mô hình BARTpho-syllable
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/897 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

dict.txt:   0%|          | 0.00/360k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.58G [00:00<?, ?B/s]

# Load data Medical

In [ ]:
# Đọc dữ liệu từ file CSV hoặc JSON
df = pd.read_csv('/content/drive/MyDrive/ChatBox/data/processed_medical.csv')  # Thay đổi đường dẫn đến tập dữ liệu của bạn

# Kiểm tra dữ liệu
df.head()




,Title,Detailed Content,Reference Link
0,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,https://www.msdmanuals.com/vi-vn/professional/...
1,rào_cản tự_nhiên chống lại nhiễm_trùng về niêm...,nhiều màng_nhầy ngập trong chất tiết có đặc_tí...,https://www.msdmanuals.com/vi-vn/professional/...
2,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,đường hô_hấp có các hệ_thống lọc đường thở trê...,https://www.msdmanuals.com/vi-vn/professional/...
3,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản trong đường tiêu_hóa bao_gồm ph ac...,https://www.msdmanuals.com/vi-vn/professional/...
4,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản về đường sinh_dục tiết_niệu bao_gồ...,https://www.msdmanuals.com/vi-vn/professional/...


In [ ]:
df.shape

(12488, 3)

In [ ]:
df['Detailed Content'] = df['Detailed Content'].str.replace('_', ' ', regex=False)

In [ ]:
df= df.head(10000)

In [ ]:
from torch.cuda.amp import autocast

def batch_summarize_contexts(contexts, batch_size=8, max_length=150, min_length=30):
    summaries = []
    for i in tqdm(range(0, len(contexts), batch_size), desc="Processing Batches"):
        batch_contexts = contexts[i:i + batch_size]
        batch_inputs = tokenizer(batch_contexts, return_tensors='pt', truncation=True, padding=True, max_length=1024)
        batch_inputs = {k: v.to(device) for k, v in batch_inputs.items()}

        with torch.no_grad():
            with autocast():  # Sử dụng FP16 để tăng tốc
                summary_ids = model.generate(
                    batch_inputs['input_ids'],
                    max_length=max_length,
                    min_length=min_length,
                    num_beams=2,  # Tăng số lượng beams để tăng độ đa dạng
                    length_penalty=1.0,
                    early_stopping=True,
                    top_k=50,  # Sử dụng Top-K sampling
                    top_p=0.95  # Sử dụng Top-p (nucleus) sampling
                )

        for summary_id in summary_ids:
            summary = tokenizer.decode(summary_id, skip_special_tokens=True)
            summaries.append(summary)

    return summaries

In [ ]:
summaries = []

# Sử dụng tqdm để hiển thị tiến độ
# Thay đổi: Truyền toàn bộ nội dung 'Detailed Content' theo batch
summaries = batch_summarize_contexts(df['Detailed Content'].tolist(), batch_size = 16, max_length=120, min_length=20)

# # Thêm các cột này vào DataFrame
# df['Summary'] = summaries

summaries
# # Kiểm tra kết quả
# df.head()

Processing Batches:   0%|          | 0/625 [00:00<?, ?it/s]<ipython-input-12-7c0548418ae8>:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Sử dụng FP16 để tăng tốc
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Processing Batches: 100%|██████████| 625/625 [41:08<00:00,  3.95s/it]


['da thường chặn các vi_sinh_vật xâm_nhập trừ khi nó bị tổn_thương ví_dụ do động_vật chân_đốt chấn_thương ống thông iv hoặc phẫu_thuật rạch các ngoại_lệ bao_gồm',
 'nhiều màng_nhầy ngập trong chất tiết có đặc_tính kháng khuẩn ví_dụ chất nhầy cổ tử_cung dịch tuyến tiền_liệt và nước_mắt có chứa lysozyme chất này chia_cắt liên_kết axit muramic trong thành tế_bào vi[',
 'đường hô_hấp có các hệ_thống lọc đường thở trên nếu các sinh_vật xâm_nhập cây phế_quản biểu mô niêm_mạc sẽ di_chuyển chúng ra khỏi phổi ho cũng giúp loại_bỏ các sinh_vật nếu các sinh[UNDERSCORE',
 'các rào_cản trong đường tiêu_hóa bao_gồm ph acid trong dạ_dày và hoạt_tính kháng khuẩn của các enzyme_tụy mật và các chất tiết đường ruột nhu_động ruột và sự đào_thải bình_thường của tế[UNDERSCORE',
 'các rào_cản về đường sinh_dục tiết_niệu bao_gồm độ dài của niệu_đạo 20 cm ở nam_giới ph acid của âm_đạo ở phụ_nữ và trạng_thái tăng trương_lực của thận và nồng[UND',
 'một_số sinh_vật ví_dụ một_số chủng phế cầu não mô cầu haemophil

In [ ]:
df['Summary'] = summaries
df

,Title,Detailed Content,Reference Link,Summary
0,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,https://www.msdmanuals.com/vi-vn/professional/...,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...
1,rào_cản tự_nhiên chống lại nhiễm_trùng về niêm...,nhiều màng_nhầy ngập trong chất tiết có đặc_tí...,https://www.msdmanuals.com/vi-vn/professional/...,nhiều màng_nhầy ngập trong chất tiết có đặc_tí...
2,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,đường hô_hấp có các hệ_thống lọc đường thở trê...,https://www.msdmanuals.com/vi-vn/professional/...,đường hô_hấp có các hệ_thống lọc đường thở trê...
3,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản trong đường tiêu_hóa bao_gồm ph ac...,https://www.msdmanuals.com/vi-vn/professional/...,các rào_cản trong đường tiêu_hóa bao_gồm ph ac...
4,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản về đường sinh_dục tiết_niệu bao_gồ...,https://www.msdmanuals.com/vi-vn/professional/...,các rào_cản về đường sinh_dục tiết_niệu bao_gồ...
...,...,...,...,...
9995,điều_trị ngáy về dụng_cụ miệng,dụng_cụ hỗ_trợ đường miệng chỉ được dùng khi n...,https://www.msdmanuals.com/vi-vn/professional/...,dụng_cụ hỗ_trợ đường miệng chỉ được dùng khi n...
9996,điều_trị ngáy về áp_lực đường thở dương liên_t...,các thiết_bị cpap duy_trì áp_suất dương không ...,https://www.msdmanuals.com/vi-vn/professional/...,các thiết_bị cpap duy_trì áp_suất dương không ...
9997,điều_trị ngáy về phẫu_thuật,bởi_vì giảm độ thông_thoáng của mũi thúc_đẩy n...,https://www.msdmanuals.com/vi-vn/professional/...,bởi_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ [UND
9998,điều_trị ngáy về thiết_bị hỗ_trợ hàm dưới,thiết_bị hỗ_trợ hàm_dưới được sử_dụng phổ_biến...,https://www.msdmanuals.com/vi-vn/professional/...,thiết_bị hỗ_trợ hàm_dưới được sử_dụng phổ_biến...


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [ ]:
df.to_csv('/content/drive/MyDrive/ChatBox/data/processed_medical2.csv', index=False, encoding='utf-8')

In [ ]:
  from rouge_score import rouge_scorer
  from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

  # Hàm tính điểm ROUGE
  def calculate_rouge_scores(summaries, references):
      scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
      scores = []
      for summary, reference in zip(summaries, references):
          score = scorer.score(reference, summary)
          scores.append(score)
      return scores

  # Hàm tính điểm BLEU
  def calculate_bleu_scores(summaries, references):
      smoothie = SmoothingFunction().method4
      scores = []
      for summary, reference in zip(summaries, references):
          # Tách từ
          reference_tokens = [reference.split()]
          summary_tokens = summary.split()
          score = sentence_bleu(reference_tokens, summary_tokens, smoothing_function=smoothie)
          scores.append(score)
      return scores


In [ ]:
# Tính điểm Rouge và BLEU cho từng bản tóm tắt
rouge_scores = calculate_rouge_scores(df['Summary'], df['Detailed Content'])
bleu_scores = calculate_bleu_scores(df['Summary'], df['Detailed Content'])

# Hiển thị kết quả
print("Rouge Scores:")
for idx, score in enumerate(rouge_scores[:5]):  # Hiển thị 5 kết quả đầu tiên
    print(f"Sample {idx + 1}:")
    print(score)

print("\nBLEU Scores:")
for idx, score in enumerate(bleu_scores[:5]):  # Hiển thị 5 kết quả đầu tiên
    print(f"Sample {idx + 1}: BLEU: {score}")

Rouge Scores:
Sample 1:
{'rouge1': Score(precision=1.0, recall=0.3708609271523179, fmeasure=0.5410628019323671), 'rouge2': Score(precision=1.0, recall=0.36666666666666664, fmeasure=0.5365853658536585), 'rougeL': Score(precision=1.0, recall=0.3708609271523179, fmeasure=0.5410628019323671)}
Sample 2:
{'rouge1': Score(precision=1.0, recall=0.3941176470588235, fmeasure=0.5654008438818565), 'rouge2': Score(precision=1.0, recall=0.3905325443786982, fmeasure=0.5617021276595745), 'rougeL': Score(precision=1.0, recall=0.3941176470588235, fmeasure=0.5654008438818565)}
Sample 3:
{'rouge1': Score(precision=0.9855072463768116, recall=0.33497536945812806, fmeasure=0.5), 'rouge2': Score(precision=0.9852941176470589, recall=0.3316831683168317, fmeasure=0.4962962962962963), 'rougeL': Score(precision=0.9855072463768116, recall=0.33497536945812806, fmeasure=0.5)}
Sample 4:
{'rouge1': Score(precision=0.9848484848484849, recall=0.20833333333333334, fmeasure=0.34391534391534395), 'rouge2': Score(precision=0

In [ ]:
df.loc[7618, 'Summary']

'triệu_chứng cứng khớp biểu_hiện bởi sự khó_khăn khi vận_động một khớp nhưng ở nhiều bệnh_nhân hạn_chế vận_động có_thể do yếu mệt_mỏi hoặc hạn_chế cố_định'

In [ ]:
def print_max_length_rows(df, columns):
    max_length_info = {}

    for column in columns:
        # Tính toán chiều dài của từng văn bản trong cột
        df[f'{column}_length'] = df[column].apply(lambda x: len(str(x)))  # Thêm cột độ dài

        # Tìm hàng có độ dài tối đa
        max_length_row = df.loc[df[f'{column}_length'].idxmax()]

        # Lưu thông tin vào từ điển
        max_length_info[column] = {
            'index': max_length_row.name,
            'length': max_length_row[f'{column}_length'],
            'value': max_length_row[column]
        }

    # In kết quả cuối cùng
    for column, info in max_length_info.items():
        print(f"Max length in '{column}':")
        print(f"Index: {info['index']}")
        print(f"Length: {info['length']}")
        print(f"Value: {info['value']}")
        print('-' * 40)

# Xác định các cột cần kiểm tra
columns_to_check = ['Title', 'Detailed Content', 'Summary']

# In hàng có độ dài tối đa trong từng cột
print_max_length_rows(df, columns_to_check)

Max length in 'Title':
Index: 99
Length: 248
Value: liều_lượng và cách sử_dụng vắc xin phòng covid 19 về đối_với các loại vắc xin ngừa covid 19 có sẵn để sử_dụng ở hoa_kỳ tất_cả thông_tin về liều_lượng và cách sử_dụng cho mọi nhóm tuổi và những người có tình_trạng đặc_biệt đều có sẵn từ cdc tại phần
----------------------------------------
Max length in 'Detailed Content':
Index: 8420
Length: 4044
Value: đái_tháo_đường có_thể là lý_do chính khiến bệnh_nhân phải nhập_viện hoặc có_thể đi kèm với các bệnh khác cần được điều_trị nội_trú tất_cả bệnh_nhân nhiễm toan ceton do đái_tháo_đường tình_trạng tăng đường_huyết tăng thẩm_thấu hoặc hạ đường_huyết kéo_dài hoặc nặng cần phải được nhập_viện bệnh_nhân với hạ đường_huyết do sulfonylureas tăng đường_huyết kiểm_soát kém hoặc biến_chứng đái_tháo đường cấp_tính nặng có_thể hưởng lợi từ nhập_viện trẻ_em và thiếu_niên với đái_tháo_đường mới khởi_phát cũng có_thể có lợi từ nhập_viện kiếm soát có_thể tồi_tệ hơn khi xuất_viện phác_đồ insulin đã được 

In [ ]:
# Đọc dữ liệu từ file CSV hoặc JSON
df = pd.read_csv('/content/drive/MyDrive/ChatBox/data/processed_medical.csv')  # Thay đổi đường dẫn đến tập dữ liệu của bạn

# Kiểm tra dữ liệu
df.head()




,Title,Detailed Content,Reference Link
0,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,https://www.msdmanuals.com/vi-vn/professional/...
1,rào_cản tự_nhiên chống lại nhiễm_trùng về niêm...,nhiều màng_nhầy ngập trong chất tiết có đặc_tí...,https://www.msdmanuals.com/vi-vn/professional/...
2,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,đường hô_hấp có các hệ_thống lọc đường thở trê...,https://www.msdmanuals.com/vi-vn/professional/...
3,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản trong đường tiêu_hóa bao_gồm ph ac...,https://www.msdmanuals.com/vi-vn/professional/...
4,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản về đường sinh_dục tiết_niệu bao_gồ...,https://www.msdmanuals.com/vi-vn/professional/...
